In [1]:
import rtdl
import torch
import torch.nn as nn
import torch.nn.functional as F
import zero

In [41]:
# Load the data
import pandas as pd

train_path = 'data/train.csv'
dev_in_path = 'data/dev_in.csv'

df_train = pd.read_csv(train_path)
df_train.head()


,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1.543321e+09,26.968800,-99.248901,2.0,0.0,dry,127.0,-17.526443,14.613571,754.263405,...,0.0,0.0,0.0,0.0,0.0,0.0,-2.600006,-2.750006,0.0,0.0
1,1.538776e+09,29.374201,-100.927002,31.0,20.0,mild temperate,297.0,41.531032,26.992143,733.117168,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.600006,17.950006,-12.0,11.0
2,1.552115e+09,22.149599,113.592003,17.0,10.0,mild temperate,-1.0,43.916531,18.842143,761.571076,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.233978,21.450006,1.0,8.0
3,1.549566e+09,34.678699,-86.684799,24.0,20.0,mild temperate,193.0,40.240955,8.303571,747.524910,...,0.0,0.0,0.0,0.0,0.0,0.0,0.059448,16.150018,-58.0,41.0
4,1.552910e+09,46.066667,41.966667,9.0,20.0,dry,90.0,30.394660,6.451429,753.168113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.400024,3.150018,18.0,92.0


In [42]:
# Remove rows with NaN in training data
print(len(df_train))
df_train = df_train.dropna()
print(len(df_train))

3129592
2965544


In [43]:
df_dev_in = pd.read_csv(dev_in_path)
df_dev_in.head()

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1539162000,-40.350000,-9.880000,11.0,10,tropical,-843.0,31.782490,10.070714,765.631228,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,0.505035,2.647577,2.0,2.0
1,1545006600,53.421299,-6.270070,4.0,10,mild temperate,67.0,-59.691521,7.005000,752.897615,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,-0.400024,1.249994,0.0,0.0
2,1540094400,-19.757700,63.361000,26.0,10,dry,6.0,35.250889,23.327143,763.115016,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,0.100006,21.050012,-1.0,1.0
3,1552611600,35.245899,47.009201,5.0,10,mild temperate,1390.0,-23.755615,3.109286,609.419333,...,1.69672,5.1653,0.0,0.000049,0.0,0.0,-1.500000,-0.349982,-12.0,81.0
4,1545631200,26.633333,118.150000,14.0,20,mild temperate,210.0,33.040438,12.172143,734.678037,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,0.102081,11.513879,-15.0,83.0


In [44]:
# Replace NaN with 0 for dev_in data
print(len(df_dev_in))
df_dev_in = df_dev_in.fillna(0)
print(len(df_dev_in))

50000
50000


In [53]:
# Set Seed
seed = 1
torch.manual_seed(seed)

In [54]:
# Preprocess into tensors
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

def get_lab_to_ind(data_df):
    '''
    Prepare a label to index map
    '''
    y_fact = set(list(data_df['fact_cwsm_class']))
    lab_to_ind = {}
    for i, lab in enumerate(y_fact):
        lab_to_ind[lab] = i
    return lab_to_ind

lab_to_ind = get_lab_to_ind(df_train)
batch_size = 256

# Train
X_train = torch.FloatTensor(np.asarray(df_train.iloc[:,6:]))
y_train = np.asarray(df_train['fact_cwsm_class'])
y_train = torch.LongTensor(np.asarray([lab_to_ind[lab] for lab in y_train]))

train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

# Dev in
X_dev_in = torch.FloatTensor(np.asarray(df_dev_in.iloc[:,6:]))
y_dev_in = df_dev_in['fact_cwsm_class']
y_dev_in = torch.LongTensor(np.asarray([lab_to_ind[lab] for lab in y_dev_in]))

dev_in_ds = TensorDataset(X_dev_in, y_dev_in)
dev_in_dl = DataLoader(dev_in_ds, batch_size=batch_size, shuffle=True)

In [55]:
# Get the device

def get_default_device():
    if torch.cuda.is_available():
        print("Got CUDA!")
        return torch.device('cuda')
    else:
        print("No CUDA found")
        return torch.device('cpu')

device = get_default_device()

No CUDA found


In [56]:
# Create the Feature Transformer Model

model = rtdl.FTTransformer.make_default(
    n_num_features=X_train.shape[1],
    cat_cardinalities=None,
    last_layer_query_idx=[-1],
    d_out=len(lab_to_ind)
)

model.to(device)

FTTransformer(
  (feature_tokenizer): FeatureTokenizer(
    (num_tokenizer): NumericalFeatureTokenizer()
  )
  (cls_token): CLSToken()
  (transformer): Transformer(
    (blocks): ModuleList(
      (0): ModuleDict(
        (attention): MultiheadAttention(
          (W_q): Linear(in_features=192, out_features=192, bias=True)
          (W_k): Linear(in_features=192, out_features=192, bias=True)
          (W_v): Linear(in_features=192, out_features=192, bias=True)
          (W_out): Linear(in_features=192, out_features=192, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (ffn): FFN(
          (linear_first): Linear(in_features=192, out_features=512, bias=True)
          (activation): ReGLU()
          (dropout): Dropout(p=0.1, inplace=False)
          (linear_second): Linear(in_features=256, out_features=192, bias=True)
        )
        (attention_residual_dropout): Dropout(p=0.0, inplace=False)
        (ffn_residual_dropout): Dropout(p=0.0, inplace=False)


In [57]:
# Create optimizer

# These params don't need to be set as we will use a default optimizer
# lr = 0.01
# weight_decay = 0.9

optimizer = (
    model.make_default_optimizer()
    if isinstance(model, rtdl.FTTransformer)
    else torch.optim.AdamW(model.parameters(), lr=lr,
                          weight_decay=weight_decay)
)

In [58]:
# Create loss function criterion
criterion = nn.CrossEntropyLoss().to(device)

In [59]:
# Create pipeline to apply model
def apply_model(model, x_num, x_cat=None):
    '''
    FTTransformer expects numerical and categorical inputs separately
    '''
    return model(x_num, x_cat) if isinstance(model, rtdl.FTTransformer) else model(x_num)

In [60]:
# Training and Evaluation Function

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy_topk(output, target, k=1):
    """Computes the topk accuracy"""
    batch_size = target.size(0)

    _, pred = torch.topk(output, k=k, dim=1, largest=True, sorted=True)

    res_total = 0
    for curr_k in range(k):
      curr_ind = pred[:,curr_k]
      num_eq = torch.eq(curr_ind, target).sum()
      acc = num_eq/len(output)
      res_total += acc
    return res_total*100

def train(train_loader, model, criterion, optimizer, epoch, device, print_freq=5):
    '''
    Run one train epoch
    '''
    losses = AverageMeter()
    accs = AverageMeter()

    # switch to train mode
    model.train()

    for i, (x, target) in enumerate(train_loader):

        x = x.to(device)
        target = target.to(device)

        # Forward pass
        logits = apply_model(model, x)
        loss = criterion(logits, target)

        # Backward pass and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure accuracy and record loss
        acc = accuracy_topk(logits.data, target)
        accs.update(acc.item(), x.size(0))
        losses.update(loss.item(), x.size(0))

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                    'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                    'Accuracy {prec.val:.3f} ({prec.avg:.3f})'.format(
                      epoch, i, len(train_loader),
                      loss=losses, prec=accs))

@torch.no_grad()
def eval(val_loader, model, criterion, device):
    '''
    Run evaluation
    '''
    losses = AverageMeter()
    accs = AverageMeter()

    # switch to eval mode
    model.eval()


    for i, (x, target) in enumerate(val_loader):

        x = x.to(device)
        target = target.to(device)

        # Forward pass
        logits = apply_model(model, x)
        loss = criterion(logits, target)

        # measure accuracy and record loss
        acc = accuracy_topk(logits.data, target)
        accs.update(acc.item(), x.size(0))
        losses.update(loss.item(), x.size(0))

    print('Dev in\t Loss ({loss.avg:.4f})\t'
            'Accuracy ({prec.avg:.3f})\n'.format(
              loss=losses, prec=accs))

In [61]:
# Train
epochs = 10
for epoch in range(epochs):

    # train for one epoch
    print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
    train(train_dl, model, criterion, optimizer, epoch, device)

    # evaluate on validation set
    eval(dev_in_dl, model, criterion, device)

current lr 1.00000e-04
Epoch: [0][0/11585]	Loss 2.4380 (2.4380)	Accuracy 1.562 (1.562)
Epoch: [0][5/11585]	Loss 1.9627 (2.1928)	Accuracy 28.125 (17.969)
Epoch: [0][10/11585]	Loss 1.6063 (1.9975)	Accuracy 35.938 (25.107)
Epoch: [0][15/11585]	Loss 1.5470 (1.8667)	Accuracy 41.016 (28.467)
Epoch: [0][20/11585]	Loss 1.4440 (1.7773)	Accuracy 36.719 (30.450)
Epoch: [0][25/11585]	Loss 1.3629 (1.7122)	Accuracy 41.797 (31.956)
Epoch: [0][30/11585]	Loss 1.3858 (1.6633)	Accuracy 39.453 (32.976)
Epoch: [0][35/11585]	Loss 1.5077 (1.6272)	Accuracy 33.203 (33.474)
Epoch: [0][40/11585]	Loss 1.4634 (1.6019)	Accuracy 38.672 (33.851)
Epoch: [0][45/11585]	Loss 1.3368 (1.5797)	Accuracy 35.938 (34.035)
Epoch: [0][50/11585]	Loss 1.3595 (1.5578)	Accuracy 35.938 (34.176)
Epoch: [0][55/11585]	Loss 1.3092 (1.5421)	Accuracy 39.062 (34.570)
Epoch: [0][60/11585]	Loss 1.4010 (1.5287)	Accuracy 39.062 (34.849)
Epoch: [0][65/11585]	Loss 1.3414 (1.5192)	Accuracy 38.672 (34.943)
Epoch: [0][70/11585]	Loss 1.3333 (1.5076)	A

Epoch: [0][605/11585]	Loss 1.1596 (1.3374)	Accuracy 49.609 (40.774)
Epoch: [0][610/11585]	Loss 1.3196 (1.3367)	Accuracy 41.797 (40.824)
Epoch: [0][615/11585]	Loss 1.1790 (1.3355)	Accuracy 48.828 (40.889)
Epoch: [0][620/11585]	Loss 1.1493 (1.3345)	Accuracy 53.516 (40.963)
Epoch: [0][625/11585]	Loss 1.1708 (1.3333)	Accuracy 49.609 (41.024)
Epoch: [0][630/11585]	Loss 1.2102 (1.3322)	Accuracy 51.172 (41.093)
Epoch: [0][635/11585]	Loss 1.2830 (1.3314)	Accuracy 42.578 (41.126)
Epoch: [0][640/11585]	Loss 1.2079 (1.3303)	Accuracy 45.312 (41.186)
Epoch: [0][645/11585]	Loss 1.1284 (1.3296)	Accuracy 52.734 (41.235)
Epoch: [0][650/11585]	Loss 1.2175 (1.3286)	Accuracy 46.484 (41.297)
Epoch: [0][655/11585]	Loss 1.1639 (1.3276)	Accuracy 47.656 (41.346)
Epoch: [0][660/11585]	Loss 1.2296 (1.3269)	Accuracy 52.344 (41.406)
Epoch: [0][665/11585]	Loss 1.1664 (1.3260)	Accuracy 50.391 (41.453)
Epoch: [0][670/11585]	Loss 1.1722 (1.3251)	Accuracy 52.344 (41.500)
Epoch: [0][675/11585]	Loss 1.2773 (1.3243)	Accur

Epoch: [0][1205/11585]	Loss 1.1763 (1.2587)	Accuracy 46.875 (45.088)
Epoch: [0][1210/11585]	Loss 1.2300 (1.2584)	Accuracy 44.141 (45.095)
Epoch: [0][1215/11585]	Loss 1.1595 (1.2581)	Accuracy 47.266 (45.105)
Epoch: [0][1220/11585]	Loss 1.1116 (1.2576)	Accuracy 57.422 (45.134)
Epoch: [0][1225/11585]	Loss 1.2680 (1.2573)	Accuracy 46.484 (45.155)
Epoch: [0][1230/11585]	Loss 1.1088 (1.2569)	Accuracy 55.859 (45.170)
Epoch: [0][1235/11585]	Loss 1.1747 (1.2565)	Accuracy 51.562 (45.186)
Epoch: [0][1240/11585]	Loss 1.1975 (1.2563)	Accuracy 48.828 (45.194)
Epoch: [0][1245/11585]	Loss 1.1203 (1.2559)	Accuracy 53.516 (45.219)
Epoch: [0][1250/11585]	Loss 1.1834 (1.2555)	Accuracy 50.781 (45.245)
Epoch: [0][1255/11585]	Loss 1.1972 (1.2552)	Accuracy 50.781 (45.258)
Epoch: [0][1260/11585]	Loss 1.1591 (1.2549)	Accuracy 50.781 (45.269)
Epoch: [0][1265/11585]	Loss 1.1948 (1.2545)	Accuracy 49.219 (45.289)
Epoch: [0][1270/11585]	Loss 1.2527 (1.2543)	Accuracy 46.875 (45.300)
Epoch: [0][1275/11585]	Loss 1.1481

KeyboardInterrupt: 